In [1]:
import numpy as np
import pandas as pd
import scipy.io as io
import os

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

torch.cuda.is_available()

True

In [3]:
import timm
from torch.utils.data.sampler import SubsetRandomSampler, WeightedRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


In [4]:
import wandb
# WANDB_NOTEBOOK_NAME = 'Train_Analysis_VRB_wandb_sweep.ipynb'
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xiaosuhu86. Use `wandb login --relogin` to force relogin


True

In [5]:
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {'goal': 'minimize', 'name': 'val_loss'},
    'parameters': 
    {
        'batch_size': {'values': [32, 64]},
        'epochs': {'values': [5, 10, 15]},
        'lr': {'values': [0.001, 0.0005, 0.0001]},
        'model': {'values': ['efficientnet_b0','efficientnet_b2','efficientnet_b4','resnet18d','resnet34d','resnet50d','resnet101d']}
     }
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project='PreBreathTraining-Pytorch-test')


Create sweep with ID: onpyoivv
Sweep URL: https://wandb.ai/xiaosuhu86/PreBreathTraining-Pytorch-test/sweeps/onpyoivv


In [6]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [7]:
class FNIRS_Dataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.targets = self.img_labels['Label']

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 4])
        image = np.float32(io.loadmat(img_path)['fnirsimg']).reshape(3,21,45)
        label = self.img_labels.iloc[idx, 5]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    return loss

def validate(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()     # Optional when not using Model Specific layer

    total_correct = 0
    total_instances = 0
    y_true=[]
    y_pred=[]

    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred,y)
        classifications = torch.argmax(model(X), dim=1)
        correct_predictions = sum(classifications==y).item()

        total_correct+=correct_predictions
        total_instances+=len(y)

        y_pred.extend(classifications.data.cpu().numpy())
        y_true.extend(y.data.cpu().numpy())
            
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"val_loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    
    acc = total_correct/total_instances

    # constant for classes
    classes = ('No_Pain','Pain')
    cf_matrix = confusion_matrix(y_true, y_pred)
    df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                     columns = [i for i in classes])
    
    TP = df_cm['Pain']['Pain']
    TN = df_cm['No_Pain']['No_Pain']

    print(f"val_acc: {acc:>7f} TP: {TP:>4f} TN: {TN:>4f}")

    return loss, acc, TP, TN

In [12]:
# Load the data with imbalanced weights
train_data=FNIRS_Dataset(
    '../Label_PreTraining.csv',
    '../PreTrainingdata/'
)

total_targets=torch.asarray(train_data.targets)
train_idx, valid_idx= train_test_split(
    np.arange(len(total_targets)), test_size=0.2, random_state=42, shuffle=True, stratify=total_targets)

train_set = torch.utils.data.Subset(train_data, train_idx)
# val_set = torch.utils.data.Subset(train_data, valid_idx)

train_sample_count = torch.tensor(
    [(total_targets[train_idx] == t).sum() for t in torch.unique(total_targets, sorted=True)])
train_weight = 1. / train_sample_count.float()
train_sample_weight = torch.tensor([train_weight[t] for t in total_targets[train_idx]])

#Creating PT data samplers
train_sampler = WeightedRandomSampler(train_sample_weight, len(train_sample_weight))
valid_sampler = SubsetRandomSampler(valid_idx)

def main():
    run = wandb.init()

    batch_size = wandb.config.batch_size

    # Create data loaders:
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, 
                                            sampler=train_sampler)
    validation_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                                    sampler=valid_sampler)

    # Model defining
    model = timm.create_model(wandb.config.model, num_classes=2, pretrained=False)
    model=model.to(device)
    
    # Other learning parameters
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.lr)
    epochs = wandb.config.epochs

    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loss = train(train_loader, model, loss_fn, optimizer)
        val_loss, val_acc, TP, TN = validate(validation_loader, model, loss_fn)

        wandb.log({
        'epoch': t+1, 
        #'train_acc': train_acc,
        'train_loss': train_loss, 
        'val_acc': val_acc, 
        'val_loss': val_loss,
        'true_pos': TP,
        'true_neg': TN,
        'model': wandb.config.model
        })

wandb.agent(sweep_id, function=main)

wandb: Agent Starting Run: v8lejojw with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	lr: 0.001
wandb: 	model: resnet50d
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1
-------------------------------
loss: 0.695969  [   32/22911]
loss: 0.717603  [ 3232/22911]
loss: 0.751577  [ 6432/22911]
loss: 0.672883  [ 9632/22911]
loss: 0.658796  [12832/22911]
loss: 0.682543  [16032/22911]
loss: 0.783420  [19232/22911]
loss: 0.738480  [22432/22911]
val_loss: 0.409262  [   32/28639]
val_loss: 0.468747  [ 3232/28639]
val_acc: 0.859637 TP: 0.082079 TN: 0.973384
Epoch 2
-------------------------------
loss: 0.612102  [   32/22911]
loss: 0.630036  [ 3232/22911]
loss: 0.585518  [ 6432/22911]
loss: 0.646199  [ 9632/22911]
loss: 0.563727  [12832/22911]
loss: 0.442669  [16032/22911]
loss: 0.515001  [19232/22911]
loss: 0.397320  [22432/22911]
val_loss: 0.339922  [   32/28639]
val_loss: 0.470569  [ 3232/28639]
val_acc: 0.777060 TP: 0.369357 TN: 0.836702
Epoch 3
-------------------------------
loss: 0.593301  [   32/22911]
loss: 0.341741  [ 3232/22911]
loss: 0.526797  [ 6432/22911]
loss: 0.529118  [ 9632/22911]
loss: 0.334991  [12832/22911]
loss: 0.354620  [16032/229

epoch,▁▃▅▆█
train_loss,▇█▄▁▂
true_neg,█▂▃▁▂
true_pos,▁▆▇██
val_acc,█▂▃▁▃
val_loss,▄▃▂█▁
epoch,5
model,resnet50d
train_loss,0.18981
true_neg,0.84531
true_pos,0.47196


wandb: Agent Starting Run: vl7xcdpd with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	lr: 0.001
wandb: 	model: resnet101d
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1
-------------------------------
loss: 0.676272  [   32/22911]
loss: 0.681165  [ 3232/22911]
loss: 0.745217  [ 6432/22911]
loss: 0.670478  [ 9632/22911]
loss: 0.669650  [12832/22911]
loss: 0.463327  [16032/22911]
loss: 0.596717  [19232/22911]
loss: 0.637388  [22432/22911]
val_loss: 0.706411  [   32/28639]
val_loss: 0.654727  [ 3232/28639]
val_acc: 0.663932 TP: 0.480164 TN: 0.690814
Epoch 2
-------------------------------
loss: 0.685681  [   32/22911]
loss: 0.715365  [ 3232/22911]
loss: 0.649001  [ 6432/22911]
loss: 0.728749  [ 9632/22911]
loss: 0.413896  [12832/22911]
loss: 0.578906  [16032/22911]
loss: 0.586901  [19232/22911]
loss: 0.666607  [22432/22911]
val_loss: 0.572693  [   32/28639]
val_loss: 0.607365  [ 3232/28639]
val_acc: 0.756459 TP: 0.361149 TN: 0.814289
Epoch 3
-------------------------------
loss: 0.505628  [   32/22911]
loss: 0.446372  [ 3232/22911]
loss: 0.636131  [ 6432/22911]
loss: 0.485960  [ 9632/22911]
loss: 0.503095  [12832/22911]
loss: 0.413918  [16032/229

...

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_loss,█▄▂▂▂▂▂▂▁▁▁▁▁▁▁
true_neg,▆█▆▁▇▅█▃▇▇▅▁▇▇▆
true_pos,▃▁▅█▃▅▂▅▃▂▅▇▃▃▃
val_acc,▆▇▆▂▆▅█▃▇▇▅▁▆█▆
val_loss,▂▂▂▂▃▁▇▁▄▂▁▂▆█▄
epoch,15
model,resnet50d
train_loss,0.02499
true_neg,0.94617
true_pos,0.15321


wandb: Agent Starting Run: m2uzl0pn with config:
wandb: 	batch_size: 64
wandb: 	epochs: 15
wandb: 	lr: 0.0001
wandb: 	model: resnet101d
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1
-------------------------------
loss: 0.701040  [   64/22911]
loss: 0.604135  [ 6464/22911]
loss: 0.619435  [12864/22911]
loss: 0.451753  [19264/22911]
val_loss: 0.496359  [   64/28639]
val_acc: 0.765189 TP: 0.285910 TN: 0.835301
Epoch 2
-------------------------------
loss: 0.526182  [   64/22911]
loss: 0.453480  [ 6464/22911]
loss: 0.179578  [12864/22911]
loss: 0.131314  [19264/22911]
val_loss: 0.914339  [   64/28639]
val_acc: 0.591655 TP: 0.608755 TN: 0.589153
Epoch 3
-------------------------------
loss: 0.304121  [   64/22911]
loss: 0.119373  [ 6464/22911]
loss: 0.043427  [12864/22911]
loss: 0.120807  [19264/22911]
val_loss: 0.573243  [   64/28639]
val_acc: 0.827514 TP: 0.239398 TN: 0.913548
Epoch 4
-------------------------------
loss: 0.055650  [   64/22911]
loss: 0.026363  [ 6464/22911]
loss: 0.043223  [12864/22911]
loss: 0.012840  [19264/22911]
val_loss: 0.628120  [   64/28639]
val_acc: 0.825943 TP: 0.216142 TN: 0.915149
Epoch 5
-------------------------------
loss: 0.

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_loss,█▃▁▁▁▂▁▁▂▁▁▁▁▁▁
true_neg,▅▁▇▇█▇██▆▇▇█▇██
true_pos,▄█▃▃▁▂▂▂▄▂▃▁▂▁▂
val_acc,▅▁▇▇█▇██▆▇▇█▇██
val_loss,▂▇▁▆▇▂▅▂▄▂█▆▆▂▄
epoch,15
model,resnet101d
train_loss,0.00338
true_neg,0.95457
true_pos,0.14911


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
